In [ ]:
import cv2
!pip install supervision
import supervision as sv
import os
!pip install ultralytics
import ultralytics
from ultralytics import YOLO
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [89]:
#cap=cv2.VideoCapture("sb-camera1-0805am-0820am.avi")
#SOURCE_VIDEO_PATH="/content/drive/MyDrive/peachtree-camera1-1245pm-0100pm.avi"
#SOURCE_VIDEO_PATH="/content/drive/MyDrive/nb-camera1-0400pm-0415pm.avi"
SOURCE_VIDEO_PATH="/content/drive/MyDrive/lankershim-camera1-0830am-0845am.avi"

In [90]:
MODEL = "yolov8x.pt"
model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [91]:
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)

video_info

VideoInfo(width=640, height=480, fps=10, total_frames=11220)

In [92]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [2, 3, 5, 7]

In [ ]:
# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=10)
# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model.track(frame, tracker="botsort.yaml", verbose=False)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, selected_classes)]
detections = byte_tracker.update_with_detections(detections)
print(detections)
print(list(detections.tracker_id))


# annotate and display frame
anotated_frame=box_annotator.annotate(scene=frame, detections=detections)

%matplotlib inline
sv.plot_image(anotated_frame, (16,16))

In [ ]:
detections.xyxy[0]

In [ ]:
# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=10)
# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4, text_thickness=4, text_scale=2)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
i=0
x=[]
y=[]
# model prediction on single frame and conversion to supervision Detections
j = 0
my_coords = {}


for frame in generator:
    print(j)
    if j < 5:
      print(my_coords)
    if j > 5000:
      break
    if True:
      results = model.track(frame, tracker="botsort.yaml", verbose=False)[0]

      # convert to Detections
      detections = sv.Detections.from_ultralytics(results)
      # only consider class id from selected_classes define above

      detections = detections[np.isin(detections.class_id, selected_classes)]

      detections = byte_tracker.update_with_detections(detections)

      '''anotated_frame=box_annotator.annotate(scene=frame, detections=detections)

      %matplotlib inline
      sv.plot_image(anotated_frame, (16,16))'''


      ids = (list(detections.tracker_id))
      for i in range(len(ids)):
        #each object get mid coordinates
        id = ids[i]
        det=detections.xyxy[i]
        x=((det[0]+det[2])/2)
        y=((det[1]+det[3])/2)

        if id not in my_coords:
          my_coords[id] = [(x,y,j)]
        else:
          my_coords[id].append((x,y,j))
    j+=1

In [81]:
final_coords = my_coords.copy()
for key in my_coords.keys():
  final_coords[key] = my_coords[key].copy()

In [82]:
for key in final_coords.keys():
  for i in range(len(final_coords[key])):
    final_coords[key][i] = (key,)+final_coords[key][i]

In [84]:
new_coords = {}
for key in final_coords.keys():
  count = 0
  final = []
  break_or_not = False
  for i in range(len(final_coords[key])):
    prev_count = final_coords[key][i-1][3]
    curr_val = final_coords[key][i][3]
    temp = []
    if curr_val != count:
      prev_x = final_coords[key][i-1][1]
      prev_y = final_coords[key][i-1][2]
      curr_x = final_coords[key][i][1]
      curr_y = final_coords[key][i][2]
      track = 1
      print((i,curr_val,count))
      for j in range(count,curr_val):
        if prev_x > curr_x:
          x = prev_x - ((abs(curr_x-prev_x)/(curr_val-prev_count))*track)
        else:
          x = prev_x + ((abs(curr_x-prev_x)/(curr_val-prev_count))*track)
        if prev_y > curr_y:
          y = prev_y - ((abs(curr_y-prev_y)/(curr_val-prev_count))*track)
        else:
          y = prev_y+ ((abs(curr_y-prev_y)/(curr_val-prev_count))*track)
        track+=1
        temp.append((key,x,y,j))
      print(temp)
      temp.append((key,final_coords[key][i][1],final_coords[key][i][2],final_coords[key][i][3]))
      count = curr_val+1
      final+=temp
    else:
      count+=1
      final.append((key,final_coords[key][i][1],final_coords[key][i][2],final_coords[key][i][3]))
  if not break_or_not:
    new_coords[key] = final


In [39]:
from scipy.spatial import distance

In [ ]:
new_coords

In [54]:
import math
for key in new_coords.keys():
  x = new_coords[key]
  new_coords[key] = [t for t in x if not any(isinstance(n, float) and math.isnan(n) for n in t)]

In [55]:
for key in new_coords.keys():
  for i in range(1,len(new_coords[key])):
    (p_x,p_y) = (new_coords[key][i-1][1],new_coords[key][i-1][2])
    (x,y) = (new_coords[key][i][1],new_coords[key][i][2])
    new_coords[key][i] = new_coords[key][i]+(distance.euclidean((p_x,p_y),(x,y)),)

In [ ]:
for key in new_coords.keys():
  print(key,len(new_coords[key]))

In [ ]:
new_coords

In [72]:
import pandas as pd
full_array = []
for key in [831]:
  temp = new_coords[key]
  print(key,len(temp))
  temp = temp[1:]
  full_array+=temp

831 198


In [ ]:
full_array

In [ ]:
a = np.asarray(full_array)
df = pd.DataFrame(a)
df = df.drop(columns=[0,3])
df = df.rename(columns={ 1: "x", 2: "y", 4: "vel"})
df

In [75]:
df.to_csv("final_I80_single.csv",header=['x','y','vel'])
#df.to_csv("final_101_single.csv",header=['x','y','vel'])
#df.to_csv("final_Peach_single.csv",header=['x','y','vel'])
#df.to_csv("final_LB_single.csv",header=['x','y','vel'])
